You need this if you are running the notebook from repo

In [1]:
import sys
sys.path.append("../")

Please note, that it is necessary to adjust this notebook according to your Matomo configuration and data structure. Please check Readme file to check the data structure expectation

In [2]:
import requests
import pandas as pd

from variatio import VariatioAnalyzer

# Configuration for Matomo API access


In [1]:
MATOMO_URL = 'https://your-matomo-domain.com'  # Replace with your Matomo domain
TOKEN_AUTH = 'your_matomo_api_token'  # Replace with your API token
SITE_ID = 'your_site_id'  # Replace with your site ID in Matomo
DATE_RANGE = '2023-01-01,2023-01-31'  # Define the date range for data extraction


In [2]:
def fetch_from_matomo(method, params):
    """
    Generic function to fetch data from the Matomo API.
    - method: Matomo API method to be called
    - params: Additional parameters for the API call
    Returns: JSON response from the API
    """
    api_params = {
        'module': 'API',
        'method': method,
        'idSite': SITE_ID,
        'period': 'range',
        'date': DATE_RANGE,
        'format': 'JSON',
        'token_auth': TOKEN_AUTH,
        **params,
    }
    response = requests.get(MATOMO_URL, params=api_params)
    return response.json()

# Fetching event data
Here we're fetching event categories, adjusting parameters as needed for your events


In [ ]:
event_data_response = fetch_from_matomo('Events.getCategory', {'secondaryDimension': 'eventName'})
event_data_df = pd.json_normalize(event_data_response)  # Normalize JSON response into a flat table
event_data_df = event_data_df[['label', 'nb_visits', 'nb_events']]  # Select relevant columns
event_data_df.columns = ['event_name', 'visits', 'events_count']  # Rename columns for clarity


# Fetching A/B test allocation events example
Assuming A/B test allocations are events with a specific naming pattern


In [ ]:
ab_test_response = fetch_from_matomo('Events.getCategory', {'secondaryDimension': 'eventAction', 'label': 'ABTest Allocation'})
ab_test_df = pd.json_normalize(ab_test_response)
ab_test_df = ab_test_df[['label', 'subtable']]  # 'subtable' might contain more detailed data requiring further processing



# Fetching user properties example (custom dimensions)
Replace 'ID_OF_YOUR_DIMENSION' with the actual ID of the custom dimension representing user properties


In [ ]:
user_properties_response = fetch_from_matomo('CustomDimensions.getCustomDimension', {'idDimension': 'ID_OF_YOUR_DIMENSION'})
user_properties_df = pd.json_normalize(user_properties_response)  # Flatten the JSON response into a DataFrame


In [6]:
analyzer = VariatioAnalyzer(event_data: event_data_df, 
                            ab_test_allocations: ab_test_df, 
                            "A", 
                            user_properties: user_properties_df)

In [7]:
# Calculate the count of 'purchase' events per user
analyzer.calculate_event_count_per_user('purchase')

# Calculate the sum of 'purchase_value' for 'purchase' events per user
analyzer.calculate_event_attribute_sum_per_user('purchase', 'purchase_value')

# Calculate the conversion rate to 'login' events
analyzer.calculate_conversion('login')

print("Metrics were calculated")

Metrics were calculated


In [9]:
analyzer.save_report("abtest_report.html")